In [42]:
# importing required modules here
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import chardet as cd

In [43]:
def find_encoding(file):
    return cd.detect(open(file, 'rb').read())['encoding']

In [44]:
encoding_for_companies_file = find_encoding('companies.txt')
encoding_for_rounds_file  = find_encoding('rounds2.csv')

In [164]:
companies = pd.read_csv('companies.txt',sep='\t',encoding=encoding_for_companies_file)
rounds2 = pd.read_csv('rounds2.csv',encoding = encoding_for_rounds_file)

In [165]:
# rounds2.drop_duplicates(subset='company_permalink',inplace=True)
companies['permalink'] = companies['permalink'].str.lower()
rounds2['company_permalink'] = rounds2['company_permalink'].str.lower()

rounds2.rename(columns={'company_permalink': 'permalink'}, inplace=True)

In [166]:
master = pd.merge(companies,rounds2,on='permalink',how='inner')
# master = pd.merge(companies,rounds2,left_on='permalink',right_on='company_permalink',how='inner')

In [168]:
master.head()

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN
2,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0
3,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0
4,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0


In [ ]:
# Total number of Investments (count)

In [ ]:
# Total amount of investment (USD)

In [ ]:
# Top Sector name (no. of investment-wise)

In [ ]:
# Second Sector name (no. of investment-wise)

In [ ]:
# Third Sector name (no. of investment-wise)

In [ ]:
# Number of investments in top sector (3)

In [ ]:
# Number of investments in second sector (4)

In [ ]:
# Number of investments in third sector (5)

In [ ]:
# For point 3 (top sector count-wise), which company received the highest investment?

In [169]:
# For point 4 (second best sector count-wise), which company received the highest investment?